In [1]:
#from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

#try:
#    fablib = fablib_manager()
                     
#    fablib.show_config()
#except Exception as e:
#    print(f"Exception: {e}")
    
import json
import traceback
import datetime 
from fabrictestbed_extensions.fablib.fablib import fablib

In [2]:
################################################################################
###################### PLease set cluster properties ###########################
################################################################################

# Number of nodes in the cluster.
num_nodes=4

# Give a cluster name
slice_name='hadoop_test'

# Site name, pick one site from the above list of resources.
site='UCSD'

# LEGACY VARIABLES
master_gpu=False


# Operation system, Linux distribution e.g. default_ubuntu_18, default_ubuntu_20, etc.
image='default_ubuntu_20'

In [4]:
node_names=[]
nic_names=[]
iface_names=[]
network_name='cluster_network'


for i in range(1,num_nodes+1):
    node_names.append("Node{0}".format(i))
    nic_names.append("Nic{0}".format(i))
    iface_names.append("iface{0}".format(i))

print(node_names)
print(nic_names)
print(iface_names)

['Node1', 'Node2', 'Node3', 'Node4']
['Nic1', 'Nic2', 'Nic3', 'Nic4']
['iface1', 'iface2', 'iface3', 'iface4']


In [5]:
try:
    slice=fablib.new_slice(name=slice_name)
    
    for i in range(num_nodes):
        if master_gpu == False and i == 0:
            node=slice.add_node(name=node_names[i],
                                site=site, 
                                cores=4,
                                ram=8,
                                disk=500,
                                image=image)
            iface_names[i]=node.add_component(model='NIC_Basic', name=node_names[i]).get_interfaces()[0]
            #node.add_component(model='NVME_P4510', name=nvme_names[i])
        
        else:
            node=slice.add_node(name=node_names[i],
                                site=site,
                                cores=4,
                                ram=8,
                                disk=500,
                                image=image)
            iface_names[i]=node.add_component(model='NIC_Basic',name=node_names[i]).get_interfaces()[0]

except Exception as e:
    print(f'Exception: {e}') 

In [6]:
try:
    net_cluster=slice.add_l2network(name=network_name, interfaces=iface_names[:])
except Exception as e:
    print(f"Exception: {e}")

In [7]:
try:
    slice.submit()
except Exception as e:
    print(f'Exception: {e}')


Retry: 10, Time: 252 sec


ID,d221c1ee-a164-47e0-bf82-666f06acfb6c
Name,hadoop_test
Lease Expiration (UTC),2024-11-05 16:40:34 +0000
Lease Start (UTC),2024-11-04 16:40:34 +0000
Project ID,a6504676-441c-4d76-ab04-5b13212df83d
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
93531383-7769-42bb-920e-3f3fad3fc7ce,Node1,4,8,500,default_ubuntu_20,qcow2,ucsd-w2.fabric-testbed.net,UCSD,ubuntu,132.249.252.137,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.137,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
7d315978-3fa6-4472-9dca-b2c7d67a63d6,Node2,4,8,500,default_ubuntu_20,qcow2,ucsd-w2.fabric-testbed.net,UCSD,ubuntu,132.249.252.190,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.190,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
749120ac-4d5f-4b1a-8b75-bc77548eec98,Node3,4,8,500,default_ubuntu_20,qcow2,ucsd-w2.fabric-testbed.net,UCSD,ubuntu,132.249.252.150,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.150,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
a6abe943-01b7-406d-b2d4-1d164e6df790,Node4,4,8,500,default_ubuntu_20,qcow2,ucsd-w2.fabric-testbed.net,UCSD,ubuntu,132.249.252.159,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.159,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
9b84b433-7bea-4f8b-b4d6-6d66e484c656,cluster_network,L2,L2Bridge,UCSD,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Node1-Node1-p1,p1,Node1,cluster_network,100,config,,2E:12:8F:2E:6F:47,enp7s0,enp7s0,fe80::2c12:8fff:fe2e:6f47,6,HundredGigE0/0/0/7
Node2-Node2-p1,p1,Node2,cluster_network,100,config,,2E:A1:B4:FF:EE:4C,enp7s0,enp7s0,fe80::2ca1:b4ff:feff:ee4c,6,HundredGigE0/0/0/7
Node3-Node3-p1,p1,Node3,cluster_network,100,config,,2E:C4:E0:E5:41:E6,enp7s0,enp7s0,fe80::2cc4:e0ff:fee5:41e6,6,HundredGigE0/0/0/7
Node4-Node4-p1,p1,Node4,cluster_network,100,config,,32:56:9F:D4:82:03,enp7s0,enp7s0,fe80::3056:9fff:fed4:8203,6,HundredGigE0/0/0/7



Time to print interfaces 270 seconds


In [8]:
slice_name = 'hadoop_test'
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

-----------  ------------------------------------
Slice Name   hadoop_test
Slice ID     d221c1ee-a164-47e0-bf82-666f06acfb6c
Slice State  StableOK
Lease End    2024-11-05 16:40:34 +0000
-----------  ------------------------------------


In [9]:

from ipaddress import IPv4Address, IPv4Network

try:
    subnet=IPv4Network("192.168.1.0/24")
    available_ips=list(subnet)[1:]
except Exception as e:
    print(f"Exception: {e}")

In [10]:
#Run this cell
#%%capture
try:
    for node in slice.get_nodes():
        node_iface=node.get_interface(network_name=network_name)
        node_IP_addr=available_ips.pop(0)
        node_iface.ip_addr_add(addr=node_IP_addr, subnet=subnet)
        
        stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}')
        _, _ = node.execute('sudo apt-get update')
        stdout, stderr = node.execute('sudo apt install net-tools')        
        stdout, stderr = node.execute(f'sudo ifconfig {node_iface.get_os_interface()} up')
        
        
except Exception as e:
    print(f"Exception: {e}")

3: enp7s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 2e:12:8f:2e:6f:47 brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.1/24 scope global enp7s0
       valid_lft forever preferred_lft forever
    inet6 fe80::2c12:8fff:fe2e:6f47/64 scope link 
       valid_lft forever preferred_lft forever
Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [3284 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security/main Translation-en [482 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security/main amd64 c-n-f Metadata [14.3 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [3222 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InReleas

In [12]:
#Run this cell
# If the ping is successful, that means the nodes are connected properly if there is any error then may need to recreate the cluster or further have to look into it. 

try:
    node1=slice.get_node(name='Node1')
    
    stdout,stderr=node1.execute(f' ping -c 3 192.168.1.3')
    print(stdout)
    print(stderr)
    
except Exception as e:
    print(f'Exception: {e}')   

PING 192.168.1.3 (192.168.1.3) 56(84) bytes of data.
64 bytes from 192.168.1.3: icmp_seq=1 ttl=64 time=0.254 ms
64 bytes from 192.168.1.3: icmp_seq=2 ttl=64 time=0.071 ms
64 bytes from 192.168.1.3: icmp_seq=3 ttl=64 time=0.088 ms

--- 192.168.1.3 ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2029ms
rtt min/avg/max/mdev = 0.071/0.137/0.254/0.082 ms
PING 192.168.1.3 (192.168.1.3) 56(84) bytes of data.
64 bytes from 192.168.1.3: icmp_seq=1 ttl=64 time=0.254 ms
64 bytes from 192.168.1.3: icmp_seq=2 ttl=64 time=0.071 ms
64 bytes from 192.168.1.3: icmp_seq=3 ttl=64 time=0.088 ms

--- 192.168.1.3 ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2029ms
rtt min/avg/max/mdev = 0.071/0.137/0.254/0.082 ms




# HADOOP INSTALLATION

### Assuming that in the above code sections a cluster is created and IP addresses are assigned to the nodes and nodes are researchable. 

# INSTRUCTIONS
* create a folder called "hadoop_scripts" in your home directory
* Copy the four scripts to the above folder only in node1 (or master node)
* Change permission of the file to execute (if needed)
* run the script "create_hadoop_cluster.sh -H"
* run the script "start_hadoop_cluster.sh"
* follow the instructions given by the script file
* enjoy!

[Tested all the scripts and notebook on 11/04, Status: Working

In [13]:
# Run this cell
# Function to create a file that contains IPs and hostnames related to it.

def append_line(file_path,text):
    with open(file_path,"a+") as file_des:
        file_des.seek(0)
        data=file_des.read(-1)
        if len(data)>0:
            file_des.write("\n")
        file_des.write(text)

In [14]:
# Run this cell

import os

if os.path.isfile('/home/fabric/work/hosts') or os.path.isfile('/home/fabric/work/ips.txt') or os.path.isfile('/home/fabric/work/workers'):
    os.system("rm /home/fabric/work/hosts")
    os.system("rm /home/fabric/work/ips.txt")
    os.system("rm /home/fabric/work/workers")    
else:
    print("does not exist")

if os.path.isfile('/home/fabric/work/gpu_ips.txt'):
    os.system("rm /home/fabric/work/gpu_ips.txt")
    
#os.system("rm /home/fabric/work/ips.txt")

In [15]:
# Run this cell

# Capturing the IP addresses, this step needs to be integrated with the IP assigning stage, coded above. Or it may stay independent.
import os

i=1
local_host="127.0.0.1 localhost"
path_to_host_file="/home/fabric/work/hosts"
path_to_ip_file="/home/fabric/work/ips.txt"
path_to_worker_ip="/home/fabric/work/workers"
path_to_gpu_ips="/home/fabric/work/gpu_ips.txt"
gpu_name="NVIDIA"

append_line(path_to_host_file,local_host)

try:
    for node in slice.get_nodes():
        stdout, stderr=node.execute("hostname -I")
        IP=stdout.split(" ")[1]
        node_name="node{0}".format(i)
        vm_names="vm{0}".format(i-1)
        append_line(path_to_ip_file,IP)
        
        stdout, stderr=node.execute("hostname")
        line=IP+" "+node_name+" "+vm_names+" "+stdout
        append_line(path_to_host_file,line)
      
    
        if(i>1):
            append_line(path_to_worker_ip,vm_names)
            #append_line(path_to_worker_ip,IP)
        
        print(line)
        print(stderr)
        
        stdout_gpu, _=node.execute('lspci | grep NVIDIA')   
        if gpu_name in stdout_gpu:
            stdout_ip, _=node.execute('hostname -I')
            ip=stdout_ip.split(" ")[1]
            append_line(path_to_gpu_ips,ip)
            #print(ip)
        
        i=i+1
except Exception as e:
    print(f"Exception: {e}")
    
print(IP)

10.20.5.154 192.168.1.1 
Node1
192.168.1.1 node1 vm0 Node1


10.20.5.26 192.168.1.2 
Node2
192.168.1.2 node2 vm1 Node2


10.20.4.233 192.168.1.3 
Node3
192.168.1.3 node3 vm2 Node3


10.20.5.99 192.168.1.4 
Node4
192.168.1.4 node4 vm3 Node4


192.168.1.4


In [16]:
# Run this cell

try:
    for node in slice.get_nodes():
        ### if you run the command twice the back up will be overwritten, a conditional block should be written
        stdout, stderr=node.execute(f'sudo cp /etc/hosts /etc/hosts_backup') 
        output_host_copy=node.upload_file(path_to_host_file,"/home/ubuntu/hosts")
        output_ip_copy=node.upload_file(path_to_ip_file,"/home/ubuntu/ips.txt")
        output_worker_copy=node.upload_file(path_to_worker_ip,"/home/ubuntu/workers")
        stdout_host_copy,stderr_host_copy=node.execute(f'sudo cp /home/ubuntu/hosts /etc/hosts')
        if os.path.isfile('/home/fabric/work/gpu_ips.txt'):
            output_gpu_copy=node.upload_file(path_to_gpu_ips,"/home/ubuntu/gpu_ips.txt")
        
        print(stderr)
        print(stderr_host_copy)
except Exception as e:
    print(f"Exception : {e}")

In [17]:
# Run this cell

import os

output=os.system('ssh-keygen -q -t rsa -N "" -f /home/fabric/work/id_rsa > /dev/null 2>&1')

In [18]:
# Run this cell

try:
    for node in slice.get_nodes():
        output_private=node.upload_file("/home/fabric/work/id_rsa","/home/ubuntu/.ssh/id_rsa")
        output_public=node.upload_file("/home/fabric/work/id_rsa.pub","/home/ubuntu/.ssh/id_rsa.pub")
        
        stdout, stderr=node.execute(f' cat /home/ubuntu/.ssh/id_rsa.pub >> /home/ubuntu/.ssh/authorized_keys')
        stdout, stderr=node.execute(f' chmod 600 /home/ubuntu/.ssh/id_rsa*')
        
        print(output_private)
        print(output_public)
        print(stderr)
        #print(stdout)
        
except Exception as e:
    print(f"Exception: {e}")

-rw-rw-r--   1 1000     1000         2622 04 Nov 16:54 ?
-rw-rw-r--   1 1000     1000          587 04 Nov 16:54 ?

-rw-rw-r--   1 1000     1000         2622 04 Nov 16:54 ?
-rw-rw-r--   1 1000     1000          587 04 Nov 16:54 ?

-rw-rw-r--   1 1000     1000         2622 04 Nov 16:54 ?
-rw-rw-r--   1 1000     1000          587 04 Nov 16:54 ?

-rw-rw-r--   1 1000     1000         2622 04 Nov 16:54 ?
-rw-rw-r--   1 1000     1000          587 04 Nov 16:54 ?

